In [19]:
import pandas as pd
import numpy as np
import json
import requests
import re
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
%matplotlib inline

#Need to use a delay between page scrapes in order to limit getting blocked by Yelp
from time import sleep

In [20]:
#ENTER SEARCH TERMS BELOW:
cuisine_type = ""
location = "San+Francisco"

#Generate URL based on search terms
base_url = "https://www.yelp.com"
search_url = f"{base_url}/search?find_desc={cuisine_type}&find_loc={location}&attrs=RestaurantsTakeOut%2CRestaurantsDelivery"

#Or manually set search_url by copying directly from Yelp Page if desired
#search_url = "https://www.yelp.com/search?find_desc=burger&find_loc
search_url

'https://www.yelp.com/search?find_desc=&find_loc=San+Francisco&attrs=RestaurantsTakeOut%2CRestaurantsDelivery'

In [21]:
star_container_class = "i-stars__09f24__M1AR7 i-stars--regular-4-half__09f24__wuUkO border-color--default__09f24__NPAKY overflow--hidden__09f24___ayzG"
price_range_class = "lemon--span__373c0__3997G text__373c0__2pB8f priceRange__373c0__2DY87 text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_ text-bullet--after__373c0__1ZHaA"
review_count_class = "reviewCount__09f24__tnBk4 css-chan6m"
next_page_class = "pagination__09f24__VRjN4 border--top__09f24__exYYb border--bottom__09f24___mg5X border-color--default__09f24__NPAKY"
search_result_class = "container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY"


In [22]:
next_page_url = search_url
page_counter = 0
business_list = []

while next_page_url:
    
    request = requests.get(next_page_url)
    soup = BeautifulSoup(request.content,'html.parser')
    
   
    search_results = soup.findAll("div", class_=search_result_class)
    print(f"Page {page_counter}, {len(search_results)-1} results {next_page_url}")
    result_counter = 1

    #
    for search_result in search_results:
        business_info = {}
        try:
            business_name_url = search_result.findAll('a', href=True)[1]
            business_info['url'] = f"https://www.yelp.com{business_name_url['href']}"
            business_info['name'] = business_name_url['name']
            business_info['biz_id'] = business_name_url['href'].split('/biz/')[1].split('?')[0]
        except:
            continue


        try:
            business_info['num_reviews'] = int(re.findall(r"[-+]?\d*\.\d+|\d+",
                                                      search_result.find(
                                                          class_=review_count_class).text )[0] )
        except:
            pass
        try:
            business_info['image_shown'] = search_result.find('img')['src']
        except:
            pass
        
        #Append business information for each search result to a list containing all businesses.
        if business_info:
            business_list.append(business_info)
            
        result_counter+=1
    
    #Set url for next page. If not found, break out of loop.
    page_counter+=10
    if page_counter >290:
        break
    next_page_url = search_url + "&start="+str(page_counter)
    

    
    #Random delay between 2 and 20 seconds to prevent getting blocked
    sleep(np.random.randint(2,20))

print(len(business_list), "businesses scraped")
#business_list

Page 0, 9 results https://www.yelp.com/search?find_desc=&find_loc=San+Francisco&attrs=RestaurantsTakeOut%2CRestaurantsDelivery
Page 10, 9 results https://www.yelp.com/search?find_desc=&find_loc=San+Francisco&attrs=RestaurantsTakeOut%2CRestaurantsDelivery&start=10
Page 20, 9 results https://www.yelp.com/search?find_desc=&find_loc=San+Francisco&attrs=RestaurantsTakeOut%2CRestaurantsDelivery&start=20
Page 30, 9 results https://www.yelp.com/search?find_desc=&find_loc=San+Francisco&attrs=RestaurantsTakeOut%2CRestaurantsDelivery&start=30
Page 40, 9 results https://www.yelp.com/search?find_desc=&find_loc=San+Francisco&attrs=RestaurantsTakeOut%2CRestaurantsDelivery&start=40
Page 50, 9 results https://www.yelp.com/search?find_desc=&find_loc=San+Francisco&attrs=RestaurantsTakeOut%2CRestaurantsDelivery&start=50
Page 60, 9 results https://www.yelp.com/search?find_desc=&find_loc=San+Francisco&attrs=RestaurantsTakeOut%2CRestaurantsDelivery&start=60
Page 70, 9 results https://www.yelp.com/search?find

In [23]:
business_info_df = pd.DataFrame(business_list)
#Drop businesses with no reviews
business_info_df.dropna(subset=['num_reviews'], inplace=True)
#Drop duplicates
business_info_df.drop(business_info_df[business_info_df.biz_id.duplicated(keep='first')].index, inplace=True)
print(len(business_info_df))
business_info_df.tail()

240


,url,name,biz_id,num_reviews,image_shown
235,https://www.yelp.com/biz/thai-idea-vegetarian-...,Thai Idea Vegetarian,thai-idea-vegetarian-san-francisco-2,768,https://s3-media0.fl.yelpcdn.com/bphoto/1itUUM...
236,https://www.yelp.com/biz/pineapple-king-bakery...,Pineapple King Bakery,pineapple-king-bakery-san-francisco-2,703,https://s3-media0.fl.yelpcdn.com/bphoto/jumiql...
237,https://www.yelp.com/biz/chloes-cafe-san-franc...,Chloe’s Cafe,chloes-cafe-san-francisco,1030,https://s3-media0.fl.yelpcdn.com/bphoto/Phl4DG...
238,https://www.yelp.com/biz/cafe-bunn-mi-san-fran...,Cafe Bunn Mi,cafe-bunn-mi-san-francisco,1344,https://s3-media0.fl.yelpcdn.com/bphoto/Or6YcT...
239,https://www.yelp.com/biz/arizmendi-bakery-san-...,Arizmendi Bakery,arizmendi-bakery-san-francisco-2,679,https://s3-media0.fl.yelpcdn.com/bphoto/O4rEX0...


In [24]:
# #CLEAN UP BUSINESS NAMES
# business_names = []
# for name in business_info_df.name:
#     name = name.replace('â\x80\x99',"\'")
#     business_names.append(name)
# business_info_df.name = business_names

In [25]:
def get_reviews(business_name, business_index, yelp_business_url, verbose=False):

    next_page_url = yelp_business_url+"?start=0"

    reviews_list = []
    page_counter=0

   
    while next_page_url:
        # if verbose:
        #    print(f"Page {page_counter}, {next_page_url}")

        
        request = requests.get(next_page_url)
        soup = BeautifulSoup(request.content,'html.parser')
        

        reviews_list.extend(get_reviews_page_v1(soup,business_name,business_index,verbose))
        
        
        if page_counter >80:
            break

        page_counter+=10
        next_page_url = yelp_business_url+"?start=" +str(page_counter)
        
    
        # if soup.find("link", attrs={'rel':'next'}):
        #     next_page_url = soup.find("link", attrs={'rel':'next'})['href']
        #     page_counter+=1
        # else:
        #     break
        
        #Random delay between 1 and 4 seconds to prevent getting blocked
        sleep(np.random.randint(1,3))
    
    return reviews_list

In [26]:
def get_reviews_page_v1(soup, business_name, business_index, verbose=False):
    
    search_result_class = "review__09f24__oHr9V border-color--default__09f24__NPAKY"
    
    #Get each review block
    reviews = soup.findAll(class_=search_result_class)
    reviews_list=[]
    skipped_review_counter=0

    for review in reviews:
        try:
            review_info = {}
            review_info["business_name"] = business_name
            review_info["business_index"] = business_index
            review_info["review"] = review.find("span", attrs={"class": "raw__09f24__T4Ezm", "lang": "en"}).text
           
            try:
                review_info["user_id"] = review.find('a')['href'].split('userid=')[1]
            except:
                None

            reviews_list.append(review_info)
        except:
            skipped_review_counter+=1
            
    if verbose:
        if skipped_review_counter!=0:
            print(f"Skipped {skipped_review_counter} reviews")

    return(reviews_list)

In [27]:
#TESTING SCRAPER FOR A SINGLE BUSINESS
index_num = 4
business_url = business_info_df.url[index_num]
business_name = business_info_df.name[index_num]
business_index = business_info_df.biz_id[index_num]

reviews_df = pd.DataFrame(get_reviews(business_name,business_index, business_url,verbose=True))

In [28]:
business_info_df.shape

(240, 5)

In [29]:
# reviews_df = pd.DataFrame()
# for index_num in range(10):
#     business_url = business_info_df.url[index_num]
#     business_name = business_info_df.name[index_num]
#     business_index = business_info_df.biz_id[index_num]

#     df = pd.DataFrame(get_reviews(business_name,business_index, business_url,verbose=True))
#     reviews_df = reviews_df.append(df)
# reviews_df

In [30]:
last_index_completed = 0
all_reviews = []
while last_index_completed < len(business_info_df):
    biz = business_info_df.iloc[last_index_completed]
    
    #Random delay between 1 and 5 seconds to try to prevent getting blocked
    sleep(np.random.randint(2,5))
    print(last_index_completed, int(biz['num_reviews']),"reviews\t", biz['name'], biz['url'], end='')
    
    #Get all reviews for the business and add to list.
    all_reviews.extend(get_reviews(biz['name'],biz['biz_id'],biz['url'])) 

    print(" completed")
    last_index_completed+=1

0 9749 reviews	 Bi-Rite Creamery https://www.yelp.com/biz/bi-rite-creamery-san-francisco completed
1 11421 reviews	 Brenda’s French Soul Food https://www.yelp.com/biz/brendas-french-soul-food-san-francisco-5 completed
2 4349 reviews	 Mitchells Ice Cream https://www.yelp.com/biz/mitchells-ice-cream-san-francisco completed
3 6379 reviews	 Hog Island Oyster https://www.yelp.com/biz/hog-island-oyster-san-francisco-2 completed
4 7059 reviews	 Burma Superstar https://www.yelp.com/biz/burma-superstar-san-francisco-2 completed
5 4762 reviews	 Kokkari Estiatorio https://www.yelp.com/biz/kokkari-estiatorio-san-francisco completed
6 3700 reviews	 Marufuku Ramen https://www.yelp.com/biz/marufuku-ramen-san-francisco-5 completed
7 4210 reviews	 Sweet Maple https://www.yelp.com/biz/sweet-maple-san-francisco completed
8 4298 reviews	 Sotto Mare Oysteria & Seafood https://www.yelp.com/biz/sotto-mare-oysteria-and-seafood-san-francisco completed
9 5706 reviews	 Tony’s Pizza Napoletana https://www.yelp.co

In [31]:
# for index_num in range(20,30):
#     business_url = business_info_df.url[index_num]
#     business_name = business_info_df.name[index_num]
#     business_index = business_info_df.biz_id[index_num]

#     df = pd.DataFrame(get_reviews(business_name,business_index, business_url,verbose=True))
#     reviews_df = reviews_df.append(df)


In [32]:
all_reviews_df = pd.DataFrame(all_reviews)
all_reviews_df

,business_name,business_index,review,user_id
0,Bi-Rite Creamery,bi-rite-creamery-san-francisco,"When I lived nearby, I didn't think much of th...",qnQnrjquwnB87jUjT2rFbg
1,Bi-Rite Creamery,bi-rite-creamery-san-francisco,Rate : 9 As a person who love all types of foo...,6hPiKG_mJhcM9vZZ6y9Q1g
2,Bi-Rite Creamery,bi-rite-creamery-san-francisco,Revisited Bi-Rite Creamery the other night for...,gLdzqo4irwK-NH19xWGJWg
3,Bi-Rite Creamery,bi-rite-creamery-san-francisco,Good ice cream right near the park. I tried t...,-iNN4s230lAs9fZTBtx1Tg
4,Bi-Rite Creamery,bi-rite-creamery-san-francisco,I first went here on a freezing August day in ...,c4bqp74_gx8vL7tEVxwUng
...,...,...,...,...
11335,Arizmendi Bakery,arizmendi-bakery-san-francisco-2,Beyond good. Only place I know in the East Bay...,q_zcWYq0pBaBGPS6zLg_BA
11336,Arizmendi Bakery,arizmendi-bakery-san-francisco-2,My recent obsession is the mexican chocolate m...,CHV3OXUzMbMDZ5A07UGO5A
11337,Arizmendi Bakery,arizmendi-bakery-san-francisco-2,"My favorite bakery in the city, hands down. Ta...",Ue6eVUvNmUJd71DY7qVksg
11338,Arizmendi Bakery,arizmendi-bakery-san-francisco-2,I love Arezmendi! We've been going here for so...,OAmS7TVcPbh3N7MZQBNeVQ


In [34]:
#business_info_df.to_csv('csv_files/business_info_all.csv',index=False)
#all_reviews_df.to_csv('reviews_SanFrancisco.csv',index=False)